In [3]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv

load_dotenv()



True

In [ ]:
#make a call to this api https://www.shiftadmin.com/vjgh/org_scheduled_shifts with a body json of start_date and end_date in the format YYYY-MM-DD
#we need to use basic auth with the username and password stored in environment variables SHIFTADMIN_USER and SHIFTADMIN_PASS
import os
SHIFTADMIN_USER = os.getenv("SHIFTADMIN_USER")
SHIFTADMIN_PASS = os.getenv("SHIFTADMIN_PASS")
def fetch_shifts(start_date, end_date):
    url = "https://www.shiftadmin.com/vjgh/org_scheduled_shifts"
    body = {
        "start_date": start_date,
        "end_date": end_date
    }
    response = requests.post(url, json=body, auth=(SHIFTADMIN_USER, SHIFTADMIN_PASS))
    response.raise_for_status()
    return response.json()  

#fetch todays shifts
today = datetime.now().date()
yesterday = today - timedelta(days=1)
tomorrow = today + timedelta(days=1)
shifts = fetch_shifts(str(yesterday), str(tomorrow))
shifts_df = pd.DataFrame(shifts)
shifts_df

In [20]:
#now make an hourly time series dataframe where the first column is called "ds" and is the datetime of each hour in the day, and then there is a column for each shift_short_name, and the value of each row is the user_id of the user assigned to that shift at that hour, or None if no user is assigned
time_index = pd.date_range(start=today, end=today + timedelta(days=2), freq='h')[:-1]
hourly_shifts_df = pd.DataFrame(index=time_index)
hourly_shifts_df.index.name = 'ds'
for _, shift in shifts_df.iterrows():
    # print(shift)
    # shift_start = pd.to_datetime(shift['shift_start'])
    # round shift start up to the next hour so e.g. 23:45 becomes 00:00
    shift_start = pd.to_datetime(shift['shift_start']).ceil('h')
    shift_end = pd.to_datetime(shift['shift_end']).ceil('h')
    shift_hours = pd.date_range(start=shift_start, end=shift_end, freq='h')[:-1]
    for hour in shift_hours:
        if hour in hourly_shifts_df.index:
            hourly_shifts_df.at[hour, shift['shift_short_name']] = shift['user_id']
hourly_shifts_df.reset_index(inplace=True)
hourly_shifts_df


,ds,WOC1,Z1,Z2,D1,R1,P1,D2,OC1,V1,...,B1,L1,E1,R2,P2,E2,A2,B2,N1,N2
0,2025-11-10 00:00:00,22.0,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-11-10 01:00:00,NaN,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-11-10 02:00:00,NaN,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-11-10 03:00:00,NaN,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-11-10 04:00:00,NaN,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2025-11-10 05:00:00,NaN,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2025-11-10 06:00:00,NaN,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2025-11-10 07:00:00,NaN,5.0,19.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2025-11-10 08:00:00,NaN,NaN,NaN,72.0,73.0,16.0,34.0,38.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2025-11-10 09:00:00,NaN,NaN,NaN,72.0,73.0,16.0,34.0,38.0,40.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#fetch the shifts starting on january 1 2021 until today, one api call per week, with 5 second delay between api calls to avoid rate limiting
all_shifts = []
start_date = datetime(2021, 1, 1).date()
end_date = today
current_start_date = start_date
while current_start_date < end_date:
    current_end_date = min(current_start_date + timedelta(days=7), end_date)
    print(f"Fetching shifts from {current_start_date} to {current_end_date}")
    shifts = fetch_shifts(str(current_start_date), str(current_end_date))
    all_shifts.extend(shifts)
    current_start_date = current_end_date + timedelta(days=1)
    import time
    time.sleep(5)  # delay to avoid rate limiting
all_shifts_df = pd.DataFrame(all_shifts)    
all_shifts_df.to_csv("all_shifts.csv", index=False)
all_shifts_df

In [23]:
all_shifts_df['shift_start'] = pd.to_datetime(all_shifts_df['shift_start'], errors='coerce')
all_shifts_df['shift_end'] = pd.to_datetime(all_shifts_df['shift_end'], errors='coerce')
all_shifts_df.to_csv("all_shifts.csv", index=False)

In [27]:
#now make an hourly time series dataframe where the first column is called "ds" and is the datetime of each hour in the day, and then there is a column for each shift_short_name, and the value of each row is the user_id of the user assigned to that shift at that hour, or None if no user is assigned
time_index = pd.date_range(start='2021-01-01', end=all_shifts_df['shift_end'].max() + timedelta(days=1), freq='h')[:-1]
hourly_shifts_df = pd.DataFrame(index=time_index)
hourly_shifts_df.index.name = 'ds'
for _, shift in all_shifts_df.iterrows():
    # print(shift)
    # shift_start = pd.to_datetime(shift['shift_start'])
    # round shift start up to the next hour so e.g. 23:45 becomes 00:00
    shift_start = pd.to_datetime(shift['shift_start']).ceil('h')
    shift_end = pd.to_datetime(shift['shift_end']).ceil('h')
    shift_hours = pd.date_range(start=shift_start, end=shift_end, freq='h')[:-1]
    for hour in shift_hours:
        if hour in hourly_shifts_df.index:
            hourly_shifts_df.at[hour, shift['shift_short_name']] = shift['user_id']
hourly_shifts_df.reset_index(inplace=True)
hourly_shifts_df


,ds,W1,X1,X3,X4,X2,WOC1,WOC2,WOC3,X5,...,N1,N2,L2,L4,H1,B1,L1,W5,L6,B2
0,2021-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-01-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-01-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42626,2025-11-12 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42627,2025-11-12 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42628,2025-11-12 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42629,2025-11-12 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
all_shifts_df.head()

In [32]:
hourly_shifts_df.iloc[8]

ds      2021-01-01 08:00:00
W1                     40.0
X1                     52.0
X3                     20.0
X4                     17.0
X2                     26.0
WOC1                    9.0
WOC2                   18.0
WOC3                   28.0
X5                     40.0
W3                      NaN
Y1                      NaN
Y3                      NaN
Y4                      NaN
Y2                      NaN
Y5                      NaN
Z1                      NaN
Z2                      NaN
D1                      NaN
R1                      NaN
P1                      NaN
D2                      NaN
OC1                     NaN
OC2                     NaN
V1                      NaN
A1                      NaN
G1                      NaN
E1                      NaN
R2                      NaN
A2                      NaN
P2                      NaN
E2                      NaN
N1                      NaN
N2                      NaN
L2                      NaN
L4                  

In [33]:
df = pd.read_csv('https://www.dropbox.com/scl/fi/s83jig4zews1xz7vhezui/allDataWithCalculatedColumns.csv?rlkey=9mm4zwaugxyj2r4ooyd39y4nl&raw=1')
df.ds = pd.to_datetime(df.ds, errors='coerce')
df.head()

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,RAZ_CONS_MORE2H,RAZ_IMCONS_MORE4H,RAZ_XRAY_MORE2H,RAZ_CT_MORE2H1,PSYCH1,PSYCH_WAITINGADM,total_tbs,vert_tbs,pod_tbs,overflow
0,2021-01-01 01:00:00,1,1,1,1,2,2,0,0,0,...,1,0,0,0,3,3,0,0,0,0
1,2021-01-01 02:00:00,2,3,1,2,3,5,1,1,0,...,1,1,0,0,3,3,3,2,1,0
2,2021-01-01 03:00:00,0,3,1,3,1,6,0,1,0,...,0,0,0,0,3,3,2,1,1,0
3,2021-01-01 04:00:00,1,4,0,3,1,7,1,2,0,...,0,0,0,0,4,3,1,0,1,0
4,2021-01-01 05:00:00,1,5,1,4,2,9,1,3,0,...,0,0,0,0,4,3,2,1,1,0


In [34]:
#join df with hourly_shifts_df on the "ds" column
merged_df = pd.merge(df, hourly_shifts_df, on='ds', how='left')
merged_df.tail()

,ds,INFLOW_STRETCHER,Infl_Stretcher_cum,INFLOW_AMBULATORY,Infl_Ambulatory_cum,Inflow_Total,Inflow_Cum_Total,INFLOW_AMBULANCES,Infl_Ambulances_cum,FLS,...,N1,N2,L2,L4,H1,B1,L1,W5,L6,B2
42506,2025-11-10 10:00:00,9,43,9,30,18,73,3,12,0,...,NaN,NaN,NaN,NaN,NaN,36.0,NaN,NaN,NaN,NaN
42507,2025-11-10 11:00:00,15,58,11,41,26,99,6,18,0,...,NaN,NaN,NaN,NaN,NaN,36.0,NaN,NaN,NaN,NaN
42508,2025-11-10 12:00:00,13,71,17,58,30,129,2,20,0,...,NaN,NaN,NaN,NaN,NaN,36.0,33.0,NaN,NaN,NaN
42509,2025-11-10 13:00:00,12,83,6,64,18,147,8,28,0,...,NaN,NaN,NaN,NaN,NaN,36.0,33.0,NaN,NaN,NaN
42510,2025-11-10 14:00:00,8,91,8,72,16,163,2,30,0,...,NaN,NaN,NaN,NaN,NaN,36.0,33.0,NaN,NaN,NaN


In [ ]:
import pandas as pd

all_shifts_from_dropbox = pd.read_csv('https://www.dropbox.com/scl/fi/yeyr2a7pj6nry8i2q3m0c/all_shifts.csv?rlkey=q1su2h8fqxfnlu7t1l2qe1w0q&raw=1')
all_shifts_from_dropbox['shift_start'] = pd.to_datetime(all_shifts_from_dropbox['shift_start'], errors='coerce')
all_shifts_from_dropbox['shift_end'] = pd.to_datetime(all_shifts_from_dropbox['shift_end'], errors='coerce')
print(all_shifts_from_dropbox.info())
all_shifts_from_dropbox.tail()

In [18]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv

load_dotenv()

#make a call to this api https://www.shiftadmin.com/vjgh/org_scheduled_shifts with a body json of start_date and end_date in the format YYYY-MM-DD
#we need to use basic auth with the username and password stored in environment variables SHIFTADMIN_USER and SHIFTADMIN_PASS

SHIFTADMIN_USER = os.getenv("SHIFTADMIN_USER")
SHIFTADMIN_PASS = os.getenv("SHIFTADMIN_PASS")
def fetch_shifts(start_date, end_date):
    url = "https://www.shiftadmin.com/vjgh/org_scheduled_shifts"
    body = {
        "start_date": start_date,
        "end_date": end_date
    }
    response = requests.post(url, json=body, auth=(SHIFTADMIN_USER, SHIFTADMIN_PASS))
    response.raise_for_status()
    return response.json()  

all_shifts_from_dropbox = pd.read_csv('https://www.dropbox.com/scl/fi/yeyr2a7pj6nry8i2q3m0c/all_shifts.csv?rlkey=q1su2h8fqxfnlu7t1l2qe1w0q&raw=1')
all_shifts_from_dropbox['shift_start'] = pd.to_datetime(all_shifts_from_dropbox['shift_start'], errors='coerce')
all_shifts_from_dropbox['shift_end'] = pd.to_datetime(all_shifts_from_dropbox['shift_end'], errors='coerce')

#fetch todays shifts
today = datetime.now().date()
lastweek = today - timedelta(days=7)
nextweek = today + timedelta(days=7)
shifts = fetch_shifts(str(lastweek), str(nextweek))
shifts_df = pd.DataFrame(shifts)
shifts_df['shift_start'] = pd.to_datetime(shifts_df['shift_start'], errors='coerce')
shifts_df['shift_end'] = pd.to_datetime(shifts_df['shift_end'], errors='coerce')

#merge with all_shifts_from_dropbox
merged_shifts_df = pd.concat([all_shifts_from_dropbox, shifts_df]).drop_duplicates(subset='scheduled_shift_id',keep='last').reset_index(drop=True)

merged_shifts_df.to_csv("all_shifts.csv", index=False)


In [ ]:
#now make an hourly time series dataframe where the first column is called "ds" and is the datetime of each hour in the day, and then there is a column for each shift_short_name, and the value of each row is the user_id of the user assigned to that shift at that hour, or None if no user is assigned
time_index = pd.date_range(start=merged_shifts_df['shift_start'].min(), end=merged_shifts_df['shift_end'].max(), freq='h')[:-1]
hourly_shifts_df = pd.DataFrame(index=time_index)
hourly_shifts_df.index.name = 'ds'
for _, shift in merged_shifts_df.iterrows():
    # print(shift)
    # shift_start = pd.to_datetime(shift['shift_start'])
    # round shift start up to the next hour so e.g. 23:45 becomes 00:00
    shift_start = pd.to_datetime(shift['shift_start']).ceil('h')
    shift_end = pd.to_datetime(shift['shift_end']).ceil('h')
    shift_hours = pd.date_range(start=shift_start, end=shift_end, freq='h')[:-1]
    for hour in shift_hours:
        if hour in hourly_shifts_df.index:
            hourly_shifts_df.at[hour, shift['shift_short_name']] = shift['first_name']+shift['last_name']
hourly_shifts_df.reset_index(inplace=True)
hourly_shifts_df


In [22]:
hourly_shifts_df.to_csv("hourly_shifts.csv", index=False)